In [1]:
import os
os.chdir('..')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


> # Forecasting future sales for Rossmann Pharmaceuticals across various stores

>>## EDA Analysis

>>> ### Load Dataset 

In [2]:
train_data_path = 'Data/train.csv'
test_data_path ='Data/test.csv'
store_data_path ='Data/store.csv'

In [3]:

from scripts.data_loader import load_data

In [4]:
train_data, test_data, store_data = load_data(train_data_path,test_data_path,store_data_path)


2025-01-02 09:52:31,498 - INFO - Loading data from file...
2025-01-02 09:52:32,364 - INFO - Train Data, Test Data and Store Data loaded with shape (1017209, 9), (41088, 8), and (1115, 10) respectively


### Check Loaded Datasets

#### Train Datasets

In [5]:
train_data.head(10)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
5,6,5,2015-07-31,5651,589,1,1,0,1
6,7,5,2015-07-31,15344,1414,1,1,0,1
7,8,5,2015-07-31,8492,833,1,1,0,1
8,9,5,2015-07-31,8565,687,1,1,0,1
9,10,5,2015-07-31,7185,681,1,1,0,1


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


#### Test Datasets

In [7]:
test_data.head(10)

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0
5,6,10,4,2015-09-17,1.0,1,0,0
6,7,11,4,2015-09-17,1.0,1,0,0
7,8,12,4,2015-09-17,1.0,1,0,0
8,9,13,4,2015-09-17,1.0,1,0,0
9,10,14,4,2015-09-17,1.0,1,0,0


In [8]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             41088 non-null  int64  
 1   Store          41088 non-null  int64  
 2   DayOfWeek      41088 non-null  int64  
 3   Date           41088 non-null  object 
 4   Open           41077 non-null  float64
 5   Promo          41088 non-null  int64  
 6   StateHoliday   41088 non-null  object 
 7   SchoolHoliday  41088 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 2.5+ MB


#### Store Dataset

In [9]:
store_data.head(10)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
5,6,a,a,310.0,12.0,2013.0,0,NaN,NaN,NaN
6,7,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN
7,8,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN
8,9,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
9,10,a,a,3160.0,9.0,2009.0,0,NaN,NaN,NaN


In [10]:
store_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


>>> ## Cleaning the data

The data cleaning process will include developing pipelines to identify and address outliers and missing data effectively.

>>>>## Check missing value

In [11]:
from scripts.data_cleaning import find_missing_values


#### Find missing values of Train Dataset

In [12]:
find_missing_values(train_data)


2025-01-02 10:22:47,627 - INFO - Finding missing values in DataFrame...
2025-01-02 10:22:47,627 - INFO - Finding missing values in DataFrame...


From 9 columns selected, there are 0 columns with missing values.


,Missing values,Percent of Total Values,DataType


#### Find Missing Values of Test Dataset

In [13]:
find_missing_values(test_data)


2025-01-02 10:23:34,565 - INFO - Finding missing values in DataFrame...
2025-01-02 10:23:34,565 - INFO - Finding missing values in DataFrame...


From 8 columns selected, there are 1 columns with missing values.


,Missing values,Percent of Total Values,DataType
Open,11,0.0,float64


#### Find missing values of Store Dataset

In [14]:
find_missing_values(store_data)


2025-01-02 10:24:01,825 - INFO - Finding missing values in DataFrame...
2025-01-02 10:24:01,825 - INFO - Finding missing values in DataFrame...


From 10 columns selected, there are 6 columns with missing values.


,Missing values,Percent of Total Values,DataType
Promo2SinceYear,544,48.8,float64
Promo2SinceWeek,544,48.8,float64
PromoInterval,544,48.8,object
CompetitionOpenSinceMonth,354,31.7,float64
CompetitionOpenSinceYear,354,31.7,float64
CompetitionDistance,3,0.3,float64


>>>>## Handling missing values

Training Data have no missing value ,Test data have one  missing value and store data have 6 missing value

In [15]:
from scripts.data_cleaning import replace_missing_values


In [16]:
test_data=replace_missing_values(test_data)


2025-01-02 10:31:25,821 - INFO - Replacing missing values...
2025-01-02 10:31:25,821 - INFO - Replacing missing values...
2025-01-02 10:31:25,829 - INFO - Replacing missing values in column 'Id' with mean: 41088
2025-01-02 10:31:25,829 - INFO - Replacing missing values in column 'Id' with mean: 41088
2025-01-02 10:31:25,831 - INFO - Replacing missing values in column 'Store' with mean: 1115
2025-01-02 10:31:25,831 - INFO - Replacing missing values in column 'Store' with mean: 1115
2025-01-02 10:31:25,835 - INFO - Replacing missing values in column 'DayOfWeek' with mean: 7
2025-01-02 10:31:25,835 - INFO - Replacing missing values in column 'DayOfWeek' with mean: 7
2025-01-02 10:31:25,838 - INFO - Replacing missing values in column 'Open' with mean: 1.0
2025-01-02 10:31:25,838 - INFO - Replacing missing values in column 'Open' with mean: 1.0
2025-01-02 10:31:25,840 - INFO - Replacing missing values in column 'Promo' with mean: 1
2025-01-02 10:31:25,840 - INFO - Replacing missing values i

In [17]:
find_missing_values(test_data)

2025-01-02 10:35:13,851 - INFO - Finding missing values in DataFrame...
2025-01-02 10:35:13,851 - INFO - Finding missing values in DataFrame...


From 8 columns selected, there are 0 columns with missing values.


,Missing values,Percent of Total Values,DataType


In [18]:
store_data=replace_missing_values(store_data)

2025-01-02 10:39:00,440 - INFO - Replacing missing values...
2025-01-02 10:39:00,440 - INFO - Replacing missing values...
2025-01-02 10:39:00,446 - INFO - Replacing missing values in column 'Store' with mean: 1115
2025-01-02 10:39:00,446 - INFO - Replacing missing values in column 'Store' with mean: 1115
2025-01-02 10:39:00,450 - INFO - Replacing missing values in column 'CompetitionDistance' with mean: 75860.0
2025-01-02 10:39:00,450 - INFO - Replacing missing values in column 'CompetitionDistance' with mean: 75860.0
2025-01-02 10:39:00,455 - INFO - Replacing missing values in column 'CompetitionOpenSinceMonth' with mean: 12.0
2025-01-02 10:39:00,455 - INFO - Replacing missing values in column 'CompetitionOpenSinceMonth' with mean: 12.0
2025-01-02 10:39:00,460 - INFO - Replacing missing values in column 'CompetitionOpenSinceYear' with mean: 2015.0
2025-01-02 10:39:00,460 - INFO - Replacing missing values in column 'CompetitionOpenSinceYear' with mean: 2015.0
2025-01-02 10:39:00,463 - 

In [19]:
find_missing_values(store_data)

2025-01-02 10:39:36,521 - INFO - Finding missing values in DataFrame...
2025-01-02 10:39:36,521 - INFO - Finding missing values in DataFrame...


From 10 columns selected, there are 0 columns with missing values.


,Missing values,Percent of Total Values,DataType


In [20]:
store_data.head(10)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,50.0,2015.0,"Jan,Apr,Jul,Oct"
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,50.0,2015.0,"Jan,Apr,Jul,Oct"
4,5,a,a,29910.0,4.0,2015.0,0,50.0,2015.0,"Jan,Apr,Jul,Oct"
5,6,a,a,310.0,12.0,2013.0,0,50.0,2015.0,"Jan,Apr,Jul,Oct"
6,7,a,c,24000.0,4.0,2013.0,0,50.0,2015.0,"Jan,Apr,Jul,Oct"
7,8,a,a,7520.0,10.0,2014.0,0,50.0,2015.0,"Jan,Apr,Jul,Oct"
8,9,a,c,2030.0,8.0,2000.0,0,50.0,2015.0,"Jan,Apr,Jul,Oct"
9,10,a,a,3160.0,9.0,2009.0,0,50.0,2015.0,"Jan,Apr,Jul,Oct"
